In [1]:
import os
from PIL import Image, ImageOps

# Input and output folders
input_folder = r"C:\Users\prita\OneDrive\Desktop\gpr project2\Category 1 - grey scale"
output_folder = r"C:\Users\prita\OneDrive\Desktop\gpr project2\Category 1 augmented output"

In [2]:
# Target resolutions
original_size = (500, 350)
horizontal_stretch = (1400, 500)  # width x height
vertical_stretch = (500, 1000)

In [3]:
# Go through each image
for idx, filename in enumerate(sorted(os.listdir(input_folder))):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        path = os.path.join(input_folder, filename)
        img = Image.open(path).convert("L")  # Grayscale

        # Resize to original target size
        img_orig = img.resize(original_size)

        # 1. Save original resized
        img_orig.save(os.path.join(output_folder, f"{idx}_orig.png"))

        # 2. Horizontal stretch
        img_hs = img.resize(horizontal_stretch)
        img_hs.save(os.path.join(output_folder, f"{idx}_horz_stretch.png"))

        # 3. Vertical stretch
        img_vs = img.resize(vertical_stretch)
        img_vs.save(os.path.join(output_folder, f"{idx}_vert_stretch.png"))

        # 4. Flips
        img_orig_flipped = ImageOps.mirror(img_orig)
        img_orig_flipped.save(os.path.join(output_folder, f"{idx}_orig_flip.png"))

        img_hs_flipped = ImageOps.mirror(img_hs)
        img_hs_flipped.save(os.path.join(output_folder, f"{idx}_horz_stretch_flip.png"))

        img_vs_flipped = ImageOps.mirror(img_vs)
        img_vs_flipped.save(os.path.join(output_folder, f"{idx}_vert_stretch_flip.png"))

print("✅ Augmentation complete! Images saved in 'category1_output'")

✅ Augmentation complete! Images saved in 'category1_output'


In [4]:
# Input and output folders
input_folder = r"C:\Users\prita\OneDrive\Desktop\gpr project2\Category 2 - blue tinted"
output_folder = r"C:\Users\prita\OneDrive\Desktop\gpr project2\Category 2 augmented output"

In [2]:
import os
from PIL import Image, ImageEnhance, ImageFilter

In [8]:
# Crop fraction (keep 80% from center)
crop_fraction = 0.8

for idx, filename in enumerate(sorted(os.listdir(input_folder))):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        img_path = os.path.join(input_folder, filename)
        img = Image.open(img_path)

        # Convert to grayscale
        img_gray = img.convert("L")

        # Get original size
        width, height = img_gray.size

        # Crop central 80%
        new_width = int(width * crop_fraction)
        new_height = int(height * crop_fraction)
        left = (width - new_width) // 2
        top = (height - new_height) // 2
        right = left + new_width
        bottom = top + new_height
        img_cropped = img_gray.crop((left, top, right, bottom))

        # Apply sharpening
        img_sharp = img_cropped.filter(ImageFilter.SHARPEN)

        # Optionally enhance contrast
        enhancer = ImageEnhance.Contrast(img_sharp)
        img_final = enhancer.enhance(1.2)  # 1.0 = original, >1 = more contrast

        # Save the processed image
        output_path = os.path.join(output_folder, f"{idx}_cropped_clear.png")
        img_final.save(output_path)

print("✅ Category 2 images cropped, sharpened, and saved in 'category2_output'")

✅ Category 2 images cropped, sharpened, and saved in 'category2_output'


In [4]:
pip install imgaug

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install albumentations --user

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import albumentations as A

In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import albumentations as A

# Set folders
input_folder = r"C:\Users\prita\OneDrive\Desktop\gpr project2\combined firt step augmented images"
train_folder = r"C:\Users\prita\OneDrive\Desktop\gpr project2\output train data"
val_folder = r"C:\Users\prita\OneDrive\Desktop\gpr project2\output validation data"
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Load all 502 grayscale images
images = []
image_names = []

for fname in sorted(os.listdir(input_folder)):
    if fname.lower().endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join(input_folder, fname)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        if img is not None:
            images.append(img)
            image_names.append(fname)

# Define Albumentations augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),                     # 50% chance to flip
    A.RandomBrightnessContrast(p=0.8, brightness_limit=0.2, contrast_limit=0.2),
    A.GaussNoise(var_limit=(10.0, 25.0), p=0.5),  # Low Gaussian noise
    A.Resize(256, 256)
])

# Augment each image 6 times
augmented_images = []
augmented_names = []

for idx, img in enumerate(images):
    for aug_idx in range(6):
        augmented = transform(image=img)
        aug_img = augmented["image"]
        augmented_images.append(aug_img)
        augmented_names.append(f"{idx}_{aug_idx}.png")

# Split into train and validation sets (90% : 10%)
X_train, X_val, y_train, y_val = train_test_split(
    augmented_images, augmented_names, test_size=0.1, random_state=42
)

# Save to disk
for img, name in zip(X_train, y_train):
    cv2.imwrite(os.path.join(train_folder, name), img)

for img, name in zip(X_val, y_val):
    cv2.imwrite(os.path.join(val_folder, name), img)

print(f"✅ Augmented {len(augmented_images)} images. Saved {len(X_train)} to train and {len(X_val)} to val.")


C:\Users\prita\AppData\Local\Temp\ipykernel_22956\3492812620.py:30: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 25.0), p=0.5),  # Low Gaussian noise


✅ Augmented 3096 images. Saved 2786 to train and 310 to val.


In [4]:
import os
import cv2

image_dir = r'yolo_data/images/validation'
label_dir = r'yolo_data/labels/validation'
os.makedirs(label_dir, exist_ok=True)

def generate_yolo_labels(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    height, width = img.shape

    # Threshold to extract bright regions (hyperbolas)
    _, thresh = cv2.threshold(img, 180, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    boxes = []

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w * h < 300:  # Skip small noise
            continue

        # Normalize to YOLO format
        x_center = (x + w / 2) / width
        y_center = (y + h / 2) / height
        norm_w = w / width
        norm_h = h / height

        boxes.append(f"0 {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}")

    return boxes

# Process all images
for img_file in os.listdir(image_dir):
    if img_file.lower().endswith(('.jpg', '.png')):
        img_path = os.path.join(image_dir, img_file)
        label_path = os.path.join(label_dir, img_file.rsplit('.', 1)[0] + '.txt')
        labels = generate_yolo_labels(img_path)

        with open(label_path, 'w') as f:
            for line in labels:
                f.write(line + '\n')


In [7]:
import os
import cv2

# === Set Paths ===
image_dir = r"yolo_data/images/validation"
label_dir = r"yolo_data/labels/validation"

# Output folder to save visualizations
output_dir = r"vis_val"
os.makedirs(output_dir, exist_ok=True)

# === Loop through images and draw bounding boxes ===
for img_file in os.listdir(image_dir):
    if not img_file.lower().endswith(('.jpg', '.png')):
        continue

    img_path = os.path.join(image_dir, img_file)
    label_path = os.path.join(label_dir, os.path.splitext(img_file)[0] + ".txt")

    # Load image
    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    # Read labels
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f.readlines():
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                class_id, x_center, y_center, box_w, box_h = map(float, parts)

                # Convert from YOLO format to pixel coordinates
                x1 = int((x_center - box_w / 2) * w)
                y1 = int((y_center - box_h / 2) * h)
                x2 = int((x_center + box_w / 2) * w)
                y2 = int((y_center + box_h / 2) * h)

                # Draw rectangle
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, f"hyperbola", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Save visualization
    out_path = os.path.join(output_dir, img_file)
    cv2.imwrite(out_path, img)

print(f"✅ Bounding box visualizations saved to: {output_dir}")


✅ Bounding box visualizations saved to: vis_val


In [6]:
import os
import cv2

image_dir = r'C:\Users\prita\OneDrive\Desktop\gpr project2\output validation data'
label_dir = r'C:\Users\prita\OneDrive\Desktop\gpr project2\output validation data labels'
os.makedirs(label_dir, exist_ok=True)

def generate_yolo_labels(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    height, width = img.shape

    # Threshold to extract bright regions (hyperbolas)
    _, thresh = cv2.threshold(img, 180, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    boxes = []

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w * h < 300:  # Skip small noise
            continue

        # Normalize to YOLO format
        x_center = (x + w / 2) / width
        y_center = (y + h / 2) / height
        norm_w = w / width
        norm_h = h / height

        boxes.append(f"0 {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}")

    return boxes

# Process all images
for img_file in os.listdir(image_dir):
    if img_file.lower().endswith(('.jpg', '.png')):
        img_path = os.path.join(image_dir, img_file)
        label_path = os.path.join(label_dir, img_file.rsplit('.', 1)[0] + '.txt')
        labels = generate_yolo_labels(img_path)

        with open(label_path, 'w') as f:
            for line in labels:
                f.write(line + '\n')


## split data

In [2]:
import os
import shutil

# 原始文件夹
src_dir = "yolo_data/images/training"
# 输出文件夹
dst_dir = "second_half_training"
os.makedirs(dst_dir, exist_ok=True)

# 获取所有文件（排序确保顺序一致）
files = sorted(os.listdir(src_dir))
n = len(files)
half_n = n // 2   # 前50%

# 复制前一半文件
for f in files[half_n:]:
    src_path = os.path.join(src_dir, f)
    dst_path = os.path.join(dst_dir, f)
    shutil.copy2(src_path, dst_path)   # copy2 会保留时间戳

print(f"✅ Done! 已复制 {half_n} 个文件到 {dst_dir}")

✅ Done! 已复制 576 个文件到 second_half_training
